In [6]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

#base e dados da base
columns = ["loc", "v(g)", "ev(g)","iv(g)", "n", "v", "l", "d","i", "e", "b", "t", "lOCode",\
                                      "loComment", "lOBlank", "locCodeAndComment", "uniq_Op", "uniq_Opnd", \
                                      "total_Op", "total_Opnd", "branchCount", "defects"]
df = pd.read_csv("../base2.csv", names=columns)
size = df.shape[1]
# Samples for training
tam = (size - 1)
feature_data = df.iloc[:, :tam]
label_data = df.iloc[:, -1]

x = feature_data.values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled, columns=df.columns[:tam])
data = pd.concat([data, label_data], axis=1)

In [41]:
all_pattern_mean = data.mean()
classes = data.iloc[:, -1].unique()

# Dispersão entre classes
sb = 0

# Dispersão intra classes
sw = 0
for l in classes:
    # Patter belongs to class type
    class_pattern = data[data.iloc[:, -1] == classes[0]]
    number_class_patterns = class_pattern.shape[0]
    class_pattern_mean = class_pattern.mean()
    means_diff = (class_pattern_mean - all_pattern_mean) **2
    sb += means_diff*number_class_patterns
    
    

loc                   0.005173
v(g)                  0.014903
ev(g)                 0.011163
iv(g)                 0.023627
n                     0.000191
v                     0.000865
l                     0.235028
d                     0.015121
i                     0.106400
e                     0.010289
b                     0.000159
t                     0.010289
lOCode                0.000985
loComment             0.062282
lOBlank               0.013522
locCodeAndComment     0.007260
uniq_Op               0.006110
uniq_Opnd             0.137841
total_Op              0.000570
total_Opnd            0.009685
branchCount           0.017602
defects              48.401913
dtype: float64